In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

## Part 1: Import the Housing data and do feature transformations

In [4]:
df= pd.read_csv('house_price_full.csv')
df.head()

,bedrooms,sqft_living,price
0,3,1340,313000
1,5,3650,2384000
2,3,1930,342000
3,3,2000,420000
4,4,1940,550000


In [5]:
X = df.copy()
# Remove target
Y = X.pop('price')

# perform a scaler transform of the input data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# perform log transformation of target variable (For Sandeep: Is this needed?)
Y = np.log(Y)

In [6]:
df_scaled = pd.DataFrame(X)
df_scaled

,0,1
0,-0.433198,-0.753258
1,1.675735,1.457330
2,-0.433198,-0.188649
3,-0.433198,-0.121661
4,0.621269,-0.179079
...,...,...
494,0.621269,0.873582
495,1.675735,2.299459
496,-0.433198,-0.724549
497,-0.433198,-0.179079


In [7]:
Y

0      12.653958
1      14.684290
2      12.742566
3      12.948010
4      13.217674
         ...    
494    13.380102
495    13.764217
496    12.128111
497    12.721886
498    12.254863
Name: price, Length: 499, dtype: float64

## Part 2: Create Model Using `keras`

![](multiple_neurons.png)

In [8]:
from tensorflow import keras

In [9]:
model = keras.Sequential(
    [
        keras.layers.Dense(
            2, activation="sigmoid", input_shape=(X.shape[-1],)
        ),
        keras.layers.Dense(1, activation="linear")
    ]
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2)                 6         
                                                                 
 dense_1 (Dense)             (None, 1)                 3         
                                                                 
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


```python
def random_init_params():
    w1 = tf.Variable(tf.random.uniform((2, 2)))
    b1 = tf.Variable(tf.random.uniform((1, 2)))
    w2 = tf.Variable(tf.random.uniform((2, 1)))
    b2 = tf.Variable(tf.random.uniform((1, 1)))
    return w1,b1,w2,b2


def forward_prop(x, w1, b1, w2, b2):
    z1 = tf.matmul(x,w1) + b1
    h1 = tf.math.sigmoid(z1)
    z2 = tf.matmul(h1,w2) + b2
    h2 = z2
    return h2
```

In [10]:
model.compile(
    optimizer=keras.optimizers.SGD(), loss="mean_squared_error"
)

```python
def train(x, y, w1, b1, w2, b2):
    y_true = y
    with tf.GradientTape() as g:
        y_pred = forward_prop(x, w1, b1, w2, b2)

        # loss
        loss = 0.5*(y_true - y_pred)** 2
    
    #Gradient calculation  
    print("**************************************************")
    print("GRADIENTS")
    print("**************************************************")
    gw1, gb1, gw2, gb2 = g.gradient(loss, [w1, b1, w2, b2])
    print(" the gradient for 1st layer weights are:\n",gw1.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 2nd layer weights are:\n",gw2.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 1st layer bias are:\n",gb1.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 2nd layer bias are:\n",gb2.numpy())
    print("--------------------------------------------------")

    # Gradient descent:
    lr=0.2
    w1.assign_sub(lr*gw1)
    b1.assign_sub(lr*gb1) 
    w2.assign_sub(lr*gw2)
    b2.assign_sub(lr*gb2)
    print("**************************************************")
    print("NEW UPDATES")
    print("**************************************************")
    print(" the updated 1st layer weights are:\n",w1.numpy())
    print("--------------------------------------------------")
    print(" the updated 2nd layer weights are:\n",w2.numpy())
    print("--------------------------------------------------")
    print(" the updated 1st layer bias are:\n",b1.numpy())
    print("--------------------------------------------------")
    print(" the updated 2nd layer bias are:\n",b2.numpy())


    return w1, b1, w2, b2,loss

```

In [11]:
model.fit(X,Y.values,epochs=10,batch_size=32)

Epoch 1/10
16/16 [==============================] - 0s 1ms/step - loss: 128.4382
Epoch 2/10
16/16 [==============================] - 0s 1ms/step - loss: 45.5742
Epoch 3/10
16/16 [==============================] - 0s 1ms/step - loss: 11.0571
Epoch 4/10
16/16 [==============================] - 0s 1ms/step - loss: 2.1058
Epoch 5/10
16/16 [==============================] - 0s 1ms/step - loss: 0.4814
Epoch 6/10
16/16 [==============================] - 0s 1ms/step - loss: 0.2049
Epoch 7/10
16/16 [==============================] - 0s 1ms/step - loss: 0.1569
Epoch 8/10
16/16 [==============================] - 0s 1ms/step - loss: 0.1459
Epoch 9/10
16/16 [==============================] - 0s 1ms/step - loss: 0.1414
Epoch 10/10
16/16 [==============================] - 0s 1ms/step - loss: 0.1390


In [12]:
model.predict(X)[:,0]

16/16 [==============================] - 0s 800us/step


array([12.779831 , 13.516021 , 13.091266 , 13.123945 , 12.953751 ,
       12.6729355, 12.94359  , 13.300837 , 13.305952 , 12.71876  ,
       12.98274  , 13.378262 , 13.266411 , 12.628178 , 13.26416  ,
       12.696337 , 12.908997 , 13.4426365, 12.797245 , 12.684111 ,
       13.22941  , 12.676369 , 13.003138 , 13.308506 , 12.766081 ,
       13.164675 , 13.278446 , 13.082695 , 13.2278805, 13.369506 ,
       12.742571 , 13.3568325, 12.890001 , 13.342241 , 13.594133 ,
       13.571299 , 12.62277  , 13.052832 , 13.105379 , 13.290333 ,
       12.914387 , 12.694649 , 12.654261 , 13.013224 , 12.867315 ,
       12.67181  , 13.110048 , 13.091266 , 12.738543 , 13.504969 ,
       13.300837 , 13.474223 , 13.408264 , 13.028215 , 12.945049 ,
       12.943314 , 13.077965 , 13.51891  , 12.768128 , 13.085131 ,
       13.504969 , 13.397873 , 13.308506 , 13.347591 , 13.378262 ,
       12.685295 , 12.951586 , 12.577219 , 13.49555  , 13.212526 ,
       13.228889 , 13.455029 , 13.119892 , 13.07699  , 12.7739